In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Function for reducing memory usage of a pandas dataframe
credit to https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65/notebook

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                #el
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else:
            #df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

<h3> <font color=blue> What is Lazy Predict

Lazy Predict helps build a lot of basic models without much code and helps understand which models works better without any parameter tuning.

<h3> <font color=blue >To install Lazy Predict

In [ ]:
!pip install lazypredict

<h3><font color=blue>Imports

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
Xy = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
Xy= reduce_mem_usage(Xy)

In [ ]:
#indexing id
Xy.set_index('id', inplace=True)

In [ ]:
# for brevity, using 10% of data
Xy_frac = Xy.sample(frac=0.10, weights = 1./Xy.groupby('target')['target'].transform('count'), random_state=3) 

In [ ]:
X,y=Xy_frac.iloc[:, :-1], Xy_frac.iloc[:, -1]

In [ ]:
del Xy, Xy_frac
gc.collect()

In [ ]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=1)

In [ ]:
#running lazy predict
clf = LazyClassifier(verbose=1,ignore_warnings=False, custom_metric=None)

In [ ]:
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

In [ ]:
#result set
# print(models)

In [ ]:
#visuale results
import matplotlib.pyplot as plt
import seaborn as sns
 
plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(y=models.index, x="ROC AUC", data=models)
plt.show